# Mise en oeuvre du filtre de Kalman étendu

## Mohamed Ben Ali

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

## Récupération des données

In [ ]:
# Lecture du tableur Excel
data = pd.read_excel(r'donnee.xlsx')

t = data["Temps"].values
y = data["signalReel"].values 
z = data["signalBruite"].values

# Définition des variables
Q = np.array([[2e-5, 0], [0, 2e-1]])
A = np.eye(2)
R = 3

nu_e = 193.28 # Hz
Te = 1/nu_e # s
nu0 = 12 # Hz

## Affichage des 300 premières valeurs du dataset

In [ ]:
nb_valeurs = 300

fig = plt.figure(figsize=(25,7))
plt.plot(t[:nb_valeurs], z[:nb_valeurs], 'bo')
plt.plot(t[:nb_valeurs], y[:nb_valeurs], 'r')
plt.title('Données mesurées')
plt.xlabel('Temps (s)')
plt.legend(["Mesures", "Signal réel"])
plt.grid()
plt.show()

## Implémentation de l'algorithme

In [ ]:
# Valeurs initiales
m0 = np.random.normal(0,1, size=(2,))
gamma0 = np.identity(2)

# Fonctions nécessaires
h = lambda x, k : x[k][0]*np.sin(2*np.pi*nu0*k*Te + x[k][1])
H = lambda x, k : np.array([np.sin(2*np.pi*nu0*k*Te + x[k][1]),
                            x[k][0]*np.cos(2*np.pi*nu0*k*Te + x[k][1])])

def extended_kalman(z, m0, gamma0, h, H):
    # Initialization
    N = len(z)
    x = np.zeros(shape = (N, 2))
    P = np.array([np.zeros(shape = (2, 2)) for _ in range(N)])
    
    x[0] = m0
    P[0] = gamma0
    
    # Loop
    for k in range(0, N-1):
        S = H(x, k).dot(P[k]).dot(H(x, k).T) + R
        K = P[k].dot(H(x, k).T)/S
        epsilon = z[k] - h(x, k)
        x[k] = x[k] + K*epsilon
        P[k] = P[k] - K*H(x, k).dot(P[k])
        
        x[k+1] = x[k]
        P[k+1] = P[k] + Q
        
    return x

## Exécution de l'algorithme sur les données

In [ ]:
x = extended_kalman(z, m0, gamma0, h, H)
y_hat = np.array([ a*np.sin(2*np.pi*nu0*k*Te + phi) for k, (a, phi) in enumerate(x)])

## Affichage du résultat sur les 300 premières valeurs

In [ ]:
nb_valeurs = 300

fig = plt.figure(figsize=(25,7))
fig.clear()
plt.plot(t[:nb_valeurs], y[:nb_valeurs], 'r', alpha=0.5)
plt.plot(t[:nb_valeurs], y_hat[:nb_valeurs], 'g')
plt.title('Estimation du signal réel')
plt.xlabel('Temps (s)')
plt.legend(["Signal réel", "Signal estimé"])
plt.grid()
plt.show()

### Commentaire

On observe que même si le signal estimé est bruité, il est tout de même proche du signal réel que l'on cherchait à estimer.

## Expérimentation sur les paramètres

On joue sur les paramètres Q et R du filtre pour observer leur influence sur l'estimation obtenue.

In [ ]:
# On fait d'abord varier R
for r in range(0,50, 10):
    R = r
    x = extended_kalman(z, m0, gamma0, h, H)
    y_hat = np.array([ a*np.sin(2*np.pi*nu0*k*Te + phi) for k, (a, phi) in enumerate(x)])
    nb_valeurs = 300

    fig = plt.figure(figsize=(25,7))
    plt.plot(t[:nb_valeurs], y[:nb_valeurs], 'r', alpha=0.5)
    plt.plot(t[:nb_valeurs], y_hat[:nb_valeurs], 'g')
    plt.title(f'R = {R}')
    plt.xlabel('Temps (s)')
    plt.legend(["Signal réel", "Signal estimé"])
    plt.grid()
    plt.show()

### Commentaire

On observe que lorsque l'on augmente R, cela fait augmenter le temps de réponse du filtre : même si l'estimation est parfois légèrement meilleure après le régime transitoire, les premières valeurs ne sont pas estimées correctement. Cela s'explique par le fait que R représente le bruit de mesure : s'il est mal estimé, notre estimation s'appuie sur un modèle inexact, qui ne représente pas bien la réalité.

In [ ]:
R = 3
# On fait ensuite varier Q
Q = np.array([[2e-5, 0], [0, 2e-1]])

for q1 in np.linspace(2e-10,2e-1, 3):
    for q2 in np.linspace(2e-10,2e-1, 3):
        Q = np.array([[q1, 0], [0, q2]])
        x = extended_kalman(z, m0, gamma0, h, H)
        y_hat = np.array([ a*np.sin(2*np.pi*nu0*k*Te + phi) for k, (a, phi) in enumerate(x)])
        nb_valeurs = 300

        fig = plt.figure(figsize=(25,7))
        plt.plot(t[:nb_valeurs], y[:nb_valeurs], 'r', alpha=0.5)
        plt.plot(t[:nb_valeurs], y_hat[:nb_valeurs], 'g')
        plt.title(f'q1 = {q1}, q2 = {q2}')
        plt.xlabel('Temps (s)')
        plt.legend(["Signal réel", "Signal estimé"])
        plt.grid()
        plt.show()

### Commentaire

On observe que les résultats sont très variables en fonction de la valeur de Q : pour certaines valeurs, l'estimation est correcte tandis que pour d'autres elle est complètement aberrante. Cela s'explique par le fait que Q représente la confiance que l'on a dans notre modèle : s'il est mal choisi, le modèle est mal représenté et on obtient une estimation qui est sujette à de grandes variations (presque aléatoire par moment).